In [24]:
# Cell 1: Kiểm tra GPU
import torch

print("Is MPS available:", torch.backends.mps.is_available())
print("Is MPS built:", torch.backends.mps.is_built())


Is MPS available: True
Is MPS built: True


In [25]:
# Cell 2: Cài đặt các thư viện cần thiết
!pip install datasets accelerate peft wandb
!pip install bitsandbytes
!pip install transformers


In [26]:
# Cell 3: Import thư viện
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainerCallback
import json
import os
import wandb
from accelerate import init_empty_weights, load_checkpoint_and_dispatch


In [ ]:
# Cell 4: Thiết lập token và đăng nhập
huggingface_token = HUGGINGFACETOKEN
login(token=huggingface_token)

wandb_api_key = WANDAPIKEY
wandb.login(key=wandb_api_key)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/nhotin/.netrc


Token is valid (permission: write).
Your token has been saved to /Users/nhotin/.cache/huggingface/token
Login successful


True

: 

In [28]:
# Cell 5: Load mô hình và tokenizer với accelerate
model_name = "Viet-Mistral/Vistral-7B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sử dụng accelerate để tải mô hình lớn và phân mảnh
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)
model = load_checkpoint_and_dispatch(
    model, model_name, device_map="auto", offload_folder="offload", offload_state_dict=True
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Cell 6: Chuẩn bị mô hình cho fine-tuning
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,  # Giá trị thấp hơn để tiết kiệm bộ nhớ
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)


NameError: name 'model' is not defined

In [ ]:
# Cell 7: Load và chuẩn bị dữ liệu
file_path = '/path/to/your/data/fine_tune_data.training_data_cleaned_af.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

dataset = Dataset.from_dict({
    "domain_name": [item["domain_name"] for item in data],
    "reference_passages": ["\n".join(item["reference_passages"]) for item in data],
    "question": [item["question"] for item in data],
    "answer": [item["answer"] for item in data]
})

print(dataset[0])


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/data/fine_tune_data.training_data_cleaned_af.json'

In [ ]:
# Cell 8: Hàm tiền xử lý
def preprocess_function(examples):
    prompts = [
        f"<s>[INST] SYSTEM: Bạn là một chatbot AI hỗ trợ hỏi đáp về Luật Doanh nghiệp Việt Nam. "
        f"Hãy đưa ra các câu trả lời chính xác và tin cậy dựa trên nội dung luật được cung cấp trong phần LAW_CONTENT. "
        f"Chỉ sử dụng thông tin có trong phần LAW_CONTENT, không tham khảo hoặc tìm kiếm thông tin từ bất kỳ nguồn nào khác. "
        f"Nếu trong phần LAW_CONTENT không có câu trả lời cho câu hỏi, hãy trả lời 'Không có thông tin. Đây là bản beta.' "
        f"DOMAIN: {domain_name} "
        f"LAW_CONTENT: {law_content} "
        f"QUESTION: {question} [/INST] "
        f"ANSWER: "
        for domain_name, law_content, question in zip(examples["domain_name"], examples["reference_passages"], examples["question"])
    ]

    model_inputs = tokenizer(prompts, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(examples["answer"], max_length=1024, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [ ]:
# Cell 9: Chia dữ liệu thành tập huấn luyện và đánh giá
tokenized_data = dataset.map(preprocess_function, batched=True)

train_data, eval_data = train_test_split(data, test_size=0.1)

train_dataset = Dataset.from_dict({
    "domain_name": [item["domain_name"] for item in train_data],
    "reference_passages": [item["reference_passages"] for item in train_data],
    "question": [item["question"] for item in train_data],
    "answer": [item["answer"] for item in train_data]
})

eval_dataset = Dataset.from_dict({
    "domain_name": [item["domain_name"] for item in eval_data],
    "reference_passages": [item["reference_passages"] for item in eval_data],
    "question": [item["question"] for item in eval_data],
    "answer": [item["answer"] for item in eval_data]
})

tokenized_train_data = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_data = eval_dataset.map(preprocess_function, batched=True)


NameError: name 'dataset' is not defined

In [ ]:
# Cell 10: Thiết lập các tham số huấn luyện
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Giảm kích thước batch
    gradient_accumulation_steps=16,  # Tăng số bước tích lũy gradient để giảm kích thước batch hiệu quả
    optim="adamw_torch",
    save_steps=10,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb",  
    run_name="nguyendntlt_vistral7B_chat_run"
)


In [ ]:
# Cell 11: Tạo Trainer và huấn luyện
from transformers import AdamW
from accelerate import Accelerator

class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not none:
            print(logs)

# Sử dụng Accelerator để chuẩn bị mô hình và optimizer
accelerator = Accelerator()
model.gradient_checkpointing_enable()  # Bật gradient checkpointing
model, optimizer = accelerator.prepare(model, AdamW(model.parameters(), lr=2e-4))

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_eval_data,
    optimizers=(optimizer, None),
    callbacks=[PrinterCallback()]
)

trainer.train()


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [ ]:
# Cell 12: Lưu mô hình đã fine-tune
from huggingface_hub import HfApi

# Tạo thư mục lưu trữ nếu chưa tồn tại
output_dir = "./fine_tuned_model"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lưu mô hình và tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Đăng nhập và lưu mô hình lên Hugging Face Hub
api = HfApi()
api.upload_folder(
    folder_path=output_dir,
    repo_id="nguyendntlt/vistral7B-chat",
    use_auth_token=huggingface_token
)


NameError: name 'model' is not defined

In [ ]:
# Cell 13: Nạp mô hình đã fine-tune từ Hugging Face Hub
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load lại mô hình và tokenizer đã fine-tune từ Hugging Face Hub
fine_tuned_model = AutoModelForCausalLM.from_pretrained("nguyendntlt/vistral7B-chat", device_map="auto", torch_dtype=torch.float16)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("nguyendntlt/vistral7B-chat")


OSError: nhotinlt/vistral7B-chat is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# Cell 14: Kiểm tra mô hình với một số câu hỏi
def generate_response(question, domain_name, law_content):
    prompt = (
        f"<s>[INST] SYSTEM: Bạn là một chatbot AI hỗ trợ hỏi đáp về Luật Doanh nghiệp Việt Nam. "
        f"Hãy đưa ra các câu trả lời chính xác và tin cậy dựa trên nội dung luật được cung cấp trong phần LAW_CONTENT. "
        f"Chỉ sử dụng thông tin có trong phần LAW_CONTENT, không tham khảo hoặc tìm kiếm thông tin từ bất kỳ nguồn nào khác. "
        f"Nếu trong phần LAW_CONTENT không có câu trả lời cho câu hỏi, hãy trả lời 'Không có thông tin. Đây là bản beta.' "
        f"DOMAIN: {domain_name} "
        f"LAW_CONTENT: {law_content} "
        f"QUESTION: {question} [/INST] "
        f"ANSWER: "
    )
    
    inputs = fine_tuned_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    inputs = {key: value.to(fine_tuned_model.device) for key, value in inputs.items()}
    
    outputs = fine_tuned_model.generate(**inputs, max_new_tokens=150)
    response = fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# Một số câu hỏi kiểm tra
questions = [
    "Điều kiện để thành lập công ty TNHH là gì?",
    "Thủ tục đăng ký doanh nghiệp bao gồm những bước nào?",
    "Các loại thuế mà doanh nghiệp cần phải nộp là gì?",
    "Quyền và nghĩa vụ của cổ đông trong công ty cổ phần như thế nào?"
]

law_content_example = (
    "Điều 7. Điều kiện để thành lập công ty TNHH\n"
    "1. Các thành viên công ty phải là cá nhân hoặc tổ chức có đủ năng lực hành vi dân sự.\n"
    "2. Công ty phải có vốn điều lệ tối thiểu là 1 tỷ đồng.\n"
    "3. Công ty phải có trụ sở chính tại Việt Nam.\n"
    "4. Công ty phải có ngành nghề kinh doanh hợp pháp.\n"
)

# Kiểm tra với các câu hỏi
for question in questions:
    print(f"Question: {question}")
    response = generate_response(question, "Luật Doanh nghiệp", law_content_example)
    print(f"Response: {response}\n")


Question: Điều kiện để thành lập công ty TNHH là gì?


NameError: name 'fine_tuned_tokenizer' is not defined